# FIXED COUPON BOND EXAMPLE TREASURY CUSIP 91282CFX4

This is based on a Bloomberg example

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import financepy

In [3]:
from financepy.utils import *
from financepy.products.bonds import *

# Define the Bond

In [4]:
issue_dt = Date(30, 11, 2022)
settle_dt = Date(6, 2, 2023)
maturity_dt = Date(30, 11, 2024)
coupon = 0.045
freq_type = FrequencyTypes.SEMI_ANNUAL
dc_type = DayCountTypes.ACT_ACT_ICMA
face = ONE_MILLION

In [5]:
bond = Bond(issue_dt, maturity_dt, coupon, freq_type, dc_type)

In [6]:
clean_price = 100+4/32+1/64  # price was quoted as 100-04+

In [7]:
clean_price

100.140625

You can get information about the bond using the print method.

In [8]:
print(bond)

OBJECT TYPE: Bond
ISSUE DATE: 30-NOV-2022
MATURITY DATE: 30-NOV-2024
COUPON (%): 4.5
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT TYPE: DayCountTypes.ACT_ACT_ICMA
EX_DIV DAYS: 0


## Bond Cash Flows

We first need to set the settlement date of the bond. 

In [9]:
bond.print_payments(settle_dt)

 31-MAY-2023      2.25000 
 30-NOV-2023      2.25000 
 31-MAY-2024      2.25000 
 30-NOV-2024    102.25000 



In [10]:
# Accrued Interest
ai = bond.accrued_interest(settle_dt, 1.0)
print(ai)

0.008406593406593407


The convention is to use these dates for yield calculations even if some fall on weekends.

## Bond Yield Measures

Current yield is the coupon over the price

In [11]:
print("Current Yield %12.6f %%" % (bond.current_yield(clean_price)*100))

Current Yield     4.493681 %


In [12]:
calcTypes = [YTMCalcType.UK_DMO, YTMCalcType.US_STREET, YTMCalcType.US_TREASURY]

Yield to maturity using different conventions

In [13]:
for ytmCalcType in calcTypes:
    y = bond.yield_to_maturity(settle_dt, clean_price, ytmCalcType)
    print("%30s %12.6f" % (ytmCalcType, y*100))

            YTMCalcType.UK_DMO     4.415114
         YTMCalcType.US_STREET     4.415114
       YTMCalcType.US_TREASURY     4.411863


In [14]:
yieldConvention = YTMCalcType.US_STREET

In [15]:
ytm = bond.yield_to_maturity(settle_dt, clean_price, yieldConvention)

In [16]:
ytm*100

4.4151139326592315

Dirty price is the clean price plus accrued interest

In [17]:
print("Dirty Price = %12.7f" % bond.dirty_price_from_ytm(settle_dt, ytm, yieldConvention))

Dirty Price =  100.9812843


In [18]:
print("Clean Price = %12.7f" % bond.clean_price_from_ytm(settle_dt, ytm, yieldConvention))

Clean Price =  100.1406250


Accrued interest is accrued from previous coupon date to settlement date

In [19]:
print("Previous coupon date is ", bond._pcd)

Previous coupon date is  30-NOV-2022


In [20]:
print("Settlement date is ", settle_dt)

Settlement date is  06-FEB-2023


The amount of accrued interest is 

In [21]:
print("Accrued = %12.2f" % bond.accrued_int)

Accrued =         0.84


This is based on the following number of days of accrual

In [22]:
print("Accrued Days = %d" % bond.accrued_days)

Accrued Days = 68


In [23]:
print("Principal = %12.2f" % bond.principal(settle_dt, ytm, face, yieldConvention))

Principal =   1009812.83


## Bond Risk Measures

The bond dollar duration is the actual derivative with respect to the yield. It is the bond price change for a 1bp drop in the yield-to-maturity divided by 1bp.

In [24]:
duration = bond.dollar_duration(settle_dt, ytm, yieldConvention)
print("Dollar Duration = ", duration)

Dollar Duration =  172.72204605944808


Modified Duration divides the dollar duration by the full price of the bond

In [25]:
modified_duration = bond.modified_duration(settle_dt, ytm, yieldConvention)
print("Modified Duration = ", modified_duration)

Modified Duration =  1.7104362178348633


Macauley Duration multiplies the dollar duration by (1+y/f) and divides by the full price

In [26]:
macauley_duration = bond.macauley_duration(settle_dt, ytm, yieldConvention)
print("Macauley Duration = ", macauley_duration)

Macauley Duration =  1.7481950717163017


Convexity is the second derivative of the bond price with respect to the yield-to-maturity

In [27]:
conv = bond.convexity_from_ytm(settle_dt, ytm, yieldConvention)
print("Convexity = ", conv)

Convexity =  0.038312539941741795


Copyright (c) 2022 Dominic O'Kane